In [176]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [177]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [178]:
tf.keras.backend.clear_session()

In [179]:
subject = 'Acre - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [180]:
data = pd.read_csv('2003_mo_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.722906,0.691320,7.285707e+06,413409.255032,9.318417,6.587220e+06,3.260
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.718028,0.691617,7.294881e+06,413607.843560,9.319990,6.590338e+06,2.191
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.713149,0.691914,7.304055e+06,413806.432089,9.321563,6.593455e+06,2.154
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.708271,0.692212,7.313229e+06,414005.020618,9.323136,6.596572e+06,2.643
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.703393,0.692509,7.322403e+06,414203.609146,9.324709,6.599689e+06,2.546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.748
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.032
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.573
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.493


In [181]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.809794,-2.186463,-1.762689,0.489108,-0.377392,-2.320347
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.812767,-2.149366,-1.742798,0.505628,-0.353105,-2.278259
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.815740,-2.112268,-1.722906,0.522147,-0.328819,-2.236171
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.818713,-2.075170,-1.703015,0.538666,-0.304532,-2.194083
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.821686,-2.038072,-1.683123,0.555185,-0.280246,-2.151995
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.241503,1.365742,1.057869,-1.253684,-1.748677,0.520808
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.236264,1.353890,1.047686,-1.237936,-1.749205,0.506927
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.231025,1.342037,1.037502,-1.222188,-1.749733,0.493046
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.225786,1.330184,1.027319,-1.206440,-1.750261,0.479164


In [182]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      4.105
1      3.898
2      4.209
3      2.994
4      3.720
       ...  
235      NaN
236      NaN
237      NaN
238      NaN
239      NaN
Name: Acre - Consumo de Cimento (t), Length: 240, dtype: float64

In [183]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.809794,-2.186463,-1.762689,0.489108,-0.377392,-2.320347
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.812767,-2.149366,-1.742798,0.505628,-0.353105,-2.278259
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.815740,-2.112268,-1.722906,0.522147,-0.328819,-2.236171
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.818713,-2.075170,-1.703015,0.538666,-0.304532,-2.194083
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.821686,-2.038072,-1.683123,0.555185,-0.280246,-2.151995
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.455828,1.391708,1.134283,-1.537220,-1.492213,0.684545
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.445046,1.388896,1.136306,-1.537199,-1.507297,0.682671
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.434265,1.386083,1.138329,-1.537178,-1.522382,0.680798
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.423484,1.383271,1.140352,-1.537157,-1.537466,0.678924


In [184]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       4.105
1       3.898
2       4.209
3       2.994
4       3.720
        ...  
157     9.081
158     9.545
159    10.130
160    14.451
161    13.218
Name: Acre - Consumo de Cimento (t), Length: 162, dtype: float64

In [185]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [186]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 13)

In [187]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.818598,0.403592,0.895819,-0.443620,0.224228,1.241058
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.846240,0.408649,0.913558,-0.464307,0.187909,1.244382
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.873883,0.413707,0.931297,-0.484993,0.151590,1.247706
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.901525,0.418765,0.949036,-0.505680,0.115270,1.251030
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.929167,0.423822,0.966775,-0.526367,0.078951,1.254354
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.241503,1.365742,1.057869,-1.253684,-1.748677,0.520808
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.236264,1.353890,1.047686,-1.237936,-1.749205,0.506927
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.231025,1.342037,1.037502,-1.222188,-1.749733,0.493046
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.225786,1.330184,1.027319,-1.206440,-1.750261,0.479164


In [188]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 13)

In [189]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [190]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(36, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [191]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [192]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3380420045, 1579068408, 60719218, 3249338389, 727130789, 1055457774, 2196295130, 2656767774, 1442110624, 1193798896, 3430624463, 3837322522, 699567826, 3488749003, 2691124665, 1713691100, 1314791459, 4040521263, 2464463111, 860364851, 50539750, 559595839, 406060555, 2290089051, 1582816419, 3861451902, 3554966954, 3199007911, 2576244950, 678088160, 4022924332, 1755504377, 52909466, 3229616154, 66471573, 2325992289, 1974867391, 2057221636, 2317366601, 2039828643, 2904056682, 51343370, 3694818986, 3813366615, 159519385, 1835041296, 1356376113, 3163765274, 3073448832, 2564456574]


Step: 0 ___________________________________________
val_loss: 25.87017059326172
winner_seed: 3380420045


Step: 1 ___________________________________________
val_loss: 1.438293695449829
winner_seed: 1579068408


Step: 2 ___________________________________________
val_loss: 20.079092025756836


Step: 3 ___________________________________________
val_loss: 8.436640739440918


Step: 4 _____________________________

In [193]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 219ms/step - loss: 158.0574 - val_loss: 77.1778
Epoch 2/10000
4/4 [==============================] - 0s 17ms/step - loss: 46.4253 - val_loss: 5.9076
Epoch 3/10000
4/4 [==============================] - 0s 16ms/step - loss: 15.0244 - val_loss: 4.0272
Epoch 4/10000
4/4 [==============================] - 0s 17ms/step - loss: 13.3754 - val_loss: 11.2012
Epoch 5/10000
4/4 [==============================] - 0s 17ms/step - loss: 13.4815 - val_loss: 5.0334
Epoch 6/10000
4/4 [==============================] - 0s 18ms/step - loss: 11.4863 - val_loss: 10.2753
Epoch 7/10000
4/4 [==============================] - 0s 18ms/step - loss: 8.9775 - val_loss: 13.9856
Epoch 8/10000
4/4 [==============================] - 0s 16ms/step - loss: 8.3950 - val_loss: 15.7428
Epoch 9/10000
4/4 [==============================] - 0s 18ms/step - loss: 7.9734 - val_loss: 19.0410
Epoch 10/10000
4/4 [==============================] - 0s 17ms/step - loss: 8.0877 - va

4/4 [==============================] - 0s 18ms/step - loss: 7.2567 - val_loss: 11.6054
Epoch 82/10000
4/4 [==============================] - 0s 17ms/step - loss: 6.8158 - val_loss: 4.7909
Epoch 83/10000
4/4 [==============================] - 0s 16ms/step - loss: 6.9780 - val_loss: 3.6298
Epoch 84/10000
4/4 [==============================] - 0s 16ms/step - loss: 7.7071 - val_loss: 12.2476
Epoch 85/10000
4/4 [==============================] - 0s 18ms/step - loss: 7.0897 - val_loss: 18.5115
Epoch 86/10000
4/4 [==============================] - 0s 19ms/step - loss: 6.8287 - val_loss: 7.4971
Epoch 87/10000
4/4 [==============================] - 0s 17ms/step - loss: 7.0639 - val_loss: 11.3757
Epoch 88/10000
4/4 [==============================] - 0s 17ms/step - loss: 6.5762 - val_loss: 16.4334
Epoch 89/10000
4/4 [==============================] - 0s 19ms/step - loss: 6.5808 - val_loss: 3.4765
Epoch 90/10000
4/4 [==============================] - 0s 20ms/step - loss: 6.9236 - val_loss: 16.5751

4/4 [==============================] - 0s 17ms/step - loss: 6.6422 - val_loss: 5.0721
Epoch 162/10000
4/4 [==============================] - 0s 16ms/step - loss: 6.6326 - val_loss: 8.7209
Epoch 163/10000
4/4 [==============================] - 0s 17ms/step - loss: 6.4382 - val_loss: 3.4378
Epoch 164/10000
4/4 [==============================] - 0s 18ms/step - loss: 6.6820 - val_loss: 4.7930
Epoch 165/10000
4/4 [==============================] - 0s 20ms/step - loss: 6.3125 - val_loss: 2.8297
Epoch 166/10000
4/4 [==============================] - 0s 18ms/step - loss: 6.2181 - val_loss: 3.7387
Epoch 167/10000
4/4 [==============================] - 0s 17ms/step - loss: 6.3813 - val_loss: 2.4877
Epoch 168/10000
4/4 [==============================] - 0s 16ms/step - loss: 6.3874 - val_loss: 3.1929
Epoch 169/10000
4/4 [==============================] - 0s 17ms/step - loss: 6.1185 - val_loss: 3.6179
Epoch 170/10000
4/4 [==============================] - 0s 17ms/step - loss: 6.2961 - val_loss: 18.

4/4 [==============================] - 0s 17ms/step - loss: 6.3009 - val_loss: 8.8909
Epoch 242/10000
4/4 [==============================] - 0s 17ms/step - loss: 6.2274 - val_loss: 6.0720
Epoch 243/10000
4/4 [==============================] - 0s 19ms/step - loss: 5.9971 - val_loss: 5.2469
Epoch 244/10000
4/4 [==============================] - 0s 19ms/step - loss: 6.1010 - val_loss: 4.9564
Epoch 245/10000
4/4 [==============================] - 0s 16ms/step - loss: 5.9104 - val_loss: 4.2335
Epoch 246/10000
4/4 [==============================] - 0s 17ms/step - loss: 5.9792 - val_loss: 3.4169
Epoch 247/10000
4/4 [==============================] - 0s 18ms/step - loss: 6.1149 - val_loss: 8.2846
Epoch 248/10000
4/4 [==============================] - 0s 19ms/step - loss: 5.9471 - val_loss: 19.3987
Epoch 249/10000
4/4 [==============================] - 0s 18ms/step - loss: 6.0746 - val_loss: 6.1009
Epoch 250/10000
4/4 [==============================] - 0s 17ms/step - loss: 6.3748 - val_loss: 11

4/4 [==============================] - 0s 18ms/step - loss: 4.7826 - val_loss: 3.5359
Epoch 322/10000
4/4 [==============================] - 0s 18ms/step - loss: 4.3951 - val_loss: 14.6176
Epoch 323/10000
4/4 [==============================] - 0s 17ms/step - loss: 4.9810 - val_loss: 20.4113
Epoch 324/10000
4/4 [==============================] - 0s 17ms/step - loss: 4.8666 - val_loss: 4.4475
Epoch 325/10000
4/4 [==============================] - 0s 18ms/step - loss: 4.3029 - val_loss: 19.7055
Epoch 326/10000
4/4 [==============================] - 0s 20ms/step - loss: 5.6939 - val_loss: 4.6698
Epoch 327/10000
4/4 [==============================] - 0s 18ms/step - loss: 5.1176 - val_loss: 4.5425
Epoch 328/10000
4/4 [==============================] - 0s 16ms/step - loss: 3.8125 - val_loss: 3.4510
Epoch 329/10000
4/4 [==============================] - 0s 16ms/step - loss: 3.9812 - val_loss: 10.0213
Epoch 330/10000
4/4 [==============================] - 0s 18ms/step - loss: 4.1172 - val_loss:

...

4/4 [==============================] - 0s 18ms/step - loss: 0.4478 - val_loss: 1.9324
Epoch 2931/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.4087 - val_loss: 1.3644
Epoch 2932/10000
4/4 [==============================] - 0s 19ms/step - loss: 0.5776 - val_loss: 2.1383
Epoch 2933/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.4675 - val_loss: 2.2234
Epoch 2934/10000
4/4 [==============================] - 0s 16ms/step - loss: 0.4329 - val_loss: 2.0201
Epoch 2935/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.4528 - val_loss: 1.8635
Epoch 2936/10000
4/4 [==============================] - 0s 19ms/step - loss: 0.4325 - val_loss: 2.1210
Epoch 2937/10000
4/4 [==============================] - 0s 20ms/step - loss: 0.4934 - val_loss: 2.0834
Epoch 2938/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.4494 - val_loss: 2.4746
Epoch 2939/10000
4/4 [==============================] - 0s 16ms/step - loss: 0.4851 - val_

4/4 [==============================] - 0s 18ms/step - loss: 0.4312 - val_loss: 3.2370
Epoch 3010/10000
4/4 [==============================] - 0s 19ms/step - loss: 0.4531 - val_loss: 2.6778
Epoch 3011/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.4646 - val_loss: 2.7580
Epoch 3012/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.4195 - val_loss: 2.8890
Epoch 3013/10000
4/4 [==============================] - 0s 16ms/step - loss: 0.4330 - val_loss: 2.7324
Epoch 3014/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.4234 - val_loss: 2.8554
Epoch 3015/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.4613 - val_loss: 2.7908
Epoch 3016/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.4015 - val_loss: 3.1352
Epoch 3017/10000
4/4 [==============================] - 0s 21ms/step - loss: 0.4739 - val_loss: 3.2691
Epoch 3018/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.4993 - val_

4/4 [==============================] - 0s 16ms/step - loss: 0.4147 - val_loss: 3.4022
Epoch 3089/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.5920 - val_loss: 3.3261
Epoch 3090/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.5361 - val_loss: 3.4752
Epoch 3091/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.4259 - val_loss: 3.5639
Epoch 3092/10000
4/4 [==============================] - 0s 19ms/step - loss: 0.3897 - val_loss: 3.4789
Epoch 3093/10000
4/4 [==============================] - 0s 19ms/step - loss: 0.4074 - val_loss: 3.4665
Epoch 3094/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.5580 - val_loss: 3.4223
Epoch 3095/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.4219 - val_loss: 3.5527
Epoch 3096/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.6601 - val_loss: 3.1936
Epoch 3097/10000
4/4 [==============================] - 0s 19ms/step - loss: 0.6372 - val_

4/4 [==============================] - 0s 18ms/step - loss: 0.3774 - val_loss: 3.8251
Epoch 3168/10000
4/4 [==============================] - 0s 16ms/step - loss: 0.4029 - val_loss: 3.7843
Epoch 3169/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.4946 - val_loss: 4.7401
Epoch 3170/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.5455 - val_loss: 3.9603
Epoch 3171/10000
4/4 [==============================] - 0s 20ms/step - loss: 0.5442 - val_loss: 3.9380
Epoch 3172/10000
4/4 [==============================] - 0s 19ms/step - loss: 0.6344 - val_loss: 3.5780
Epoch 3173/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.6513 - val_loss: 3.6035
Epoch 3174/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.5493 - val_loss: 3.9780
Epoch 3175/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.5531 - val_loss: 3.7238
Epoch 3176/10000
4/4 [==============================] - 0s 19ms/step - loss: 0.4844 - val_

4/4 [==============================] - 0s 17ms/step - loss: 0.3413 - val_loss: 3.3826
Epoch 3247/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.4367 - val_loss: 3.4330
Epoch 3248/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.3874 - val_loss: 3.3991
Epoch 3249/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.4300 - val_loss: 3.3818
Epoch 3250/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.4101 - val_loss: 3.3724
Epoch 3251/10000
4/4 [==============================] - 0s 18ms/step - loss: 0.3434 - val_loss: 3.3667
Epoch 3252/10000
4/4 [==============================] - 0s 19ms/step - loss: 0.4051 - val_loss: 3.6036
Epoch 3253/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.3933 - val_loss: 3.2270
Epoch 3254/10000
4/4 [==============================] - 0s 16ms/step - loss: 0.4837 - val_loss: 3.7916
Epoch 3255/10000
4/4 [==============================] - 0s 17ms/step - loss: 0.7517 - val_

In [194]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [195]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,11.179091,10.709495,10.483058,10.746663,10.845981,10.899626,10.923664,10.94705,11.019748,11.106559,11.175641,11.677714,11.711164,12.271613,12.44108,12.425797,12.305598,12.044409,12.129979,12.34847,12.225724,12.283661,12.420517,12.416793,12.540403,12.529288,12.989557,14.150483,16.21121,18.155252
Target,13.552,15.826,14.869,14.734,12.754,11.925,13.114,8.92,9.724,9.737,10.136,14.403,12.871,15.749,14.269,15.367,14.595,9.811,11.188,10.381,8.08,11.154,12.508,12.126,14.496,16.723,15.253,16.531,15.206,10.16
Error,2.372909,5.116506,4.385942,3.987337,1.908019,1.025374,2.190336,2.02705,1.295748,1.369558,1.039641,2.725286,1.159836,3.477386,1.82792,2.941202,2.289402,2.233409,0.941979,1.96747,4.145724,1.129661,0.087483,0.290792,1.955597,4.193711,2.263443,2.380517,1.00521,7.995253


In [196]:
display(mae)
display(mape)

2.3909903

0.1908212

In [197]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [198]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined